# CS145: Project 2 | Traffic Fatalaties Across the United States

## Collaborators:
Please list the names and SUNet IDs of your collaborators below:
* *Leon MacAlister, leonmac*
* *Jonathan Affeld, jaffeld*

## Project Overview

---
**MAIN QUESTION**

Are Traffic Fatalities consistent across the United States or are there specific states with Higher or Lower fatality rates? What factors might contribute to these regional variations?

**PROJECT OVERVIEW**

We are using the nhtsa_traffic_fatalities data set to find which factors significantly increase the severity of fatal crashes such as atmospheric conditions, emergency response times and how it differs across state. To do this we will try to find out how a wide range of factors along with their combinations, along with a way to measure the severity of a fatal accident. Moreover we want to find which are the most common factors that are involved within car accidents. 

**SUPPLEMENTARY QUESTIONS**
- Are the number of deaths in a crash determined by the emergency response time? Perhaps the severity of a crash may be very contingent upon the effectiveness of emergency services or proximity to hospitals where emergency care can be provided. And does emergency response time on average differ from state to state?

- What times/days are the most fatal, and what proportion of those crashes during those times involve drunk drivers. We wonder if there are more drunk drivers in certain states? And if hours with most fatal crashes vary across these states?

- What atmospheric conditions cause a higher percentage of severe crashes? And do differences in unfavourable conditions across states increase number of fatal crashes in a statistically significant way?

- Does maneuvering increase or decrease the number of fatalities in a crash? Or is it more of an indicator of whether or not someone is drunk/impaired?
Being alert may cause someone to swerve as opposed to being drunk/asleep, we also wonder if swerving leads to more or less deaths on the road.

- Which contributing factors and their combinations are most common in severe/fatal crashes (find various permutations of contributing factors)? We want to know if a combination of factors such as being impaired + being distracted result or being impaired + swerving will result in more severe crashes 



---


# Analysis of Dataset

In [ ]:
!pip install pandas
!pip install pandas-gbq
!pip install matplotlib
!pip install plotly
!pip install plotly-express

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly_express as px

In [ ]:
# Run this cell to authenticate yourself to BigQuery
from google.oauth2 import service_account
key_path = './cs145-project2-406000-9a59fc7c0b3d.json'
credential = service_account.Credentials.from_service_account_file(key_path)

In [ ]:
# Initialize BiqQuery client
from google.cloud import bigquery
%load_ext google.cloud.bigquery
%env GOOGLE_APPLICATION_CREDENTIALS=$key_path
project_id = "cs145-project2-406000"
client = bigquery.Client(credentials=credential, project=project_id)

### Taste of the Dataset (just using 2016)

ACCIDENTS (~30mb each for 6 years - 180mb)
State_name (primary)
Number_of_motor_vehicles_in_transport_mvit (primary)
Number_of_parked_working_vehicles (primary)
Number_of_persons_in_motor_vehicles_in_transport_mvit (primary)
Number_of_persons_not_in_motor_vehicles_in_transport_mvit (primary)
City_name (primary)
Day_name (primary)
Month_of_crash_name (primary)
Day_of_week_name (primary)
Hour_of_crash_name (primary)
Route_signing_name (primary)
Land_use (primary) 
Land_use_name (primary) *Very vague e.g. rural, urban etc.
First_harmful_event_name (primary) 
Work_zone (primary)
Work_zone_name (primary)
Relation_to_trafficway_name (primary)
Light_condition_name (primary)
Atmospheric_conditions_name (primary)
Minute_of_arrival_at_scene (primary) *MAKE A NEW FEATURE DIFFERENCE BETWEEN CRASH TIME AND ARRIVAL AT SCENE
Minute_of_ems_arrival_at_hospital_name (primary) *MAKE A NEW FEATURE DIFFERENCE BETWEEN CRASH TIME AND ARRIVAL AT SCENE
Number_of_fatalities (primary)
Number_of_drunk_drivers (primary)

ACCIDENTS is the main table of the dataset. It has a high level description of each crash e.g. when, where, and some description of the actual crash. Moreover it gives us a handful of other useful columns such as atmospheric condition which indicates if it was raining, cloudy etc. Another column like light_condition gives information on how bright or dark it is on a numerical scale (e.g. dark and lighted, dusk, daylight etc.). However the more interesting columns are the crash time and arrival of emergency services at the scene - which may give us a lot of insight into how the quickness of emergency response can help reduce the number of fatalities in a car accident. Also note that the consecutive number is the code for the crash, which includes all cars involved in that particular accident. We will also use this identification number to join all the other tables to make the dataset more robust. There are some data inconsistencies from year to year, but we are mainly choosing to exclude 2015 as there are a couple of columns missing from this dataset that we will be using. So we will be drawing quantitative insights from five years of data, not six.

Moreover we need to be mindful that since there are multiple victims or participants in a fatal car accident, that there may be several rows in other tables below that refer to the same crash but for a different vehicle. For example in a three way crash, DISTRACT will include an entry for all three drivers. This is the same for other tables as well like MANEUVER etc. Therefore it is crucial we keep this in mind when we query data or train a model.

In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2016`
LIMIT 5


VISION (~3mb over 6 years - 18mb)
Consecutive_number  (secondary)
Vehicle_number  (secondary)
Drivers_vision_obscured_by_name (primary)

An add on where we can join consecutive numbers to the accidents table. This delineates the number and type of vision obstruction that occurred and contributed to the car accident e.g. “reflected glare, bright sunlight”. 

In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `bigquery-public-data.nhtsa_traffic_fatalities. vision_2016`
LIMIT 5


DISTRACT (~3mb over 6 years - 18mb)
Consecutive_number  (secondary)
Vehicle_number  (secondary)
Driver_distracted_by_name (primary)

This is a simple add on where we can join consecutive numbers to the accidents table. Here the driver_distracted_by_name will give us information on whether or not the driver was distracted. If not it will give us “not reported” or “unknown”. Or if the driver was distracted, will give us the source of distraction e.g. “distracted by an outside person”. 


In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `bigquery-public-data.nhtsa_traffic_fatalities. distract_2016`
LIMIT 5


DRIMPAIR (~3.5mb over 6 years - 21mb)
Consecutive_number  (secondary)
Vehicle_number  (secondary)
Condition_impairment_at_time_of_crash_driver_name (primary)

This is another add on table where we can join consecutive numbers to the accidents table. Here the table will give us information on whether or not the driver had a physical impairment that may have contributed to the crash. If not it will give us “None/Apparently Normal” or “unknown”. 

In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `bigquery-public-data.nhtsa_traffic_fatalities. drimpair_2016`
LIMIT 5


MANEUVER (~3mb over 6 years - 18mb)
Consecutive_number  (secondary)
Vehicle_number  (secondary)
Driver_maneuvered_to_avoid_name (primary)

Another add on where we can join consecutive numbers to the accidents table. Here the table will give us information on whether or not the driver maneuvered to avoid an object or car e.g. “motor vehicle”


In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `bigquery-public-data.nhtsa_traffic_fatalities. maneuver_2016`
LIMIT 5

VIOLATION (~2.5mb over 6 years - 15mb)
Consecutive_number  (secondary)
Vehicle_number  (secondary)
Violations_charged_name (primary)

An add on where we can join consecutive numbers to the accidents table. This delineates the number and type of road rule violations that occurred e.g. “no license plate”. 

In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `bigquery-public-data.nhtsa_traffic_fatalities. violation_2016`
LIMIT 5

NMCRASH (~<1mb over 6 years - 5mb)
Consecutive_number  (secondary)
Vehicle_number  (secondary)
Non_motorist_contributing_circumstances_name (primary)

Another add on where we can join consecutive numbers to the accidents table. Here the table will give us information on how a non_motorist may have contributed to a crash. For example if a pedestrian was wearing “dark clothes” or not.

In [ ]:
%%bigquery --project $project_id
SELECT *
FROM `bigquery-public-data.nhtsa_traffic_fatalities. nmcrash_2016`
LIMIT 5

**Issues/Inconsistencies with the dataset** 
- CEVENT, VSOE, VEVENT seem to be describing very similar things, all including sequence of event coding for each crash (consec. number). Very detailed and descriptive data with not a lot of matching entries, hard to do much quantitative analysis on this.Quite unrelated to our question as this covers when crash actually occurs instead of contributing factors 
- For the contributing factor tables - although they all have encoded types, severity of impairment, distraction etc. is often not delineated and therefore hard to implement more gradiation in queries 
- Inconsistent type for hours, and therefore had to change it to string  
- Consecutive number provides unique ID codes for each accident, however they are reused from year to year, therefore we had to append year to these codes to ensure all accidents are unique in the aggregated table we made
- No data for the state of Indiana 


**Pivoting Away From Using Other Tables Outside of Accidents**
- We initially joined several tables on consecutive number to ACCIDENTS including DRIMPAIR, DISTRACT, VIOLATIONS, VISION, NMIMPAIR etc. Consecutive number gives us the ID of a specific crash.
- As aforementioned to overcome the fact that IDs get re-used from year to year, we appended the year number to the consecutive number to ensure that all accidents are unique 
-Moreover for certain tables such as nmcrash, drimpair, nmimpair, and violation we must account for the fact that if there are multiple participants within an accident there will be multiple entries for a specific accident id number. 
 e.g. if there is a head-on collision, there will be row entries for both drivers in drimpair table under the same accident ID that states if either driver was physically impaired or not. Therefore when joining tables that have many entries for the same accident we get several duplicates and permutations that add extreme bulk to our dataset. This resulted in certain specific crashes having thousands of different entries after joining on multiple different tables due to the sheer amount of permutations. 
- Hence we decided to cut down on the number of feature tables to join on to ACCIDENTS to just two: DISTRACT and DRIMPAIR. However we continued to encounter more data consistency issues.
- In the end we decided to use just the ACCIDENTS table and join across years as it already had a swathe of interesting features.





----
# Analysis of Dataset
----

## Investigate how we should create our car severity equation

Below is a query that aims to investigate the number of accidents with different numbers of vehicles in transport. As we can see here, most accidents involve one car. And number of accidents increase as number of motor vehicles in transport decreases. Ultimately we want to see if number of vehicles in transport involved in accident should be used to determine crash severity. 

In [ ]:
%%bigquery --project $project_id

SELECT number_of_motor_vehicles_in_transport_mvit, COUNT(number_of_motor_vehicles_in_transport_mvit) AS count
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2016`
GROUP BY number_of_motor_vehicles_in_transport_mvit
ORDER BY number_of_motor_vehicles_in_transport_mvit

Below is a query that aims to investigate the number of accidents with different numbers of vehicles that are parked.

In [ ]:
%%bigquery --project $project_id

SELECT number_of_parked_working_vehicles, COUNT(number_of_parked_working_vehicles) AS count
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2016`
GROUP BY number_of_parked_working_vehicles
ORDER BY number_of_parked_working_vehicles

We also wanted to see how accidents were stratified by number of fatalities. Below we noticed that like number of vehicles involved (which makes sense), number of fatalities in an accident is mostly 1. As such we see that most fatal accidents in the US do not involve more than just the driver.

In [ ]:
%%bigquery --project $project_id

SELECT number_of_fatalities, COUNT(number_of_fatalities) AS count
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2016`
GROUP BY number_of_fatalities
ORDER BY number_of_fatalities

When determining the label, we were disappointed to find that the data set only includes accidents with fatalities. We could do so much more with a dataset that included accidents with and without fatalities. We found that in the vast majority of the dataset, entries only had one fatality, thus determining severity purely based on number of fatalities would not be realistic for a model.

 Just so we intitially decided to artificially make a label variable that estimates the severity of a crash and creates a nice distribution between low severity and high severity. We let the severity = LOG(number_of_motor_vehicles_in_transport_mvit) + 2 * LOG(number_of_parked_working_vehicles + 1) + number_of_fatalities = 1. This is because vehicles that are parked are less likely to have occupants. Moreover we think the number of injuries and damage caused will greatly increase with number of moving vehicles in the crash. Finally we weight the number of fatalities in an accident the most for obvious reasons. Below we see that the data is still slightly biased towards "low severity fatal accidents". 

Nevertheless we decided the number here was too abstract, and therefore opted to classify severe fatal crashes as any accident with more than one car or fatality. And in the end we chose our label to simply be number of fatalities within an accident - as we think this is the most simple way to determine the severity of fatal car accidents.

In [ ]:
%%bigquery --project $project_id

SELECT IF(number_of_motor_vehicles_in_transport_mvit + number_of_parked_working_vehicles > 1 OR number_of_fatalities > 1, 1, 0) AS label, COUNT(*) AS count
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2016`
GROUP BY label

## Aggregate the Data we want into one Table

- We unioned all the accident tables from year to year (2016-2020) to make a master accidents table. As a result we got ~150,000 fatal accidents 
- We made number of fatalities the label
- As aforementioned we noticed that the consecutive numbers (the unique ID code for a specific accident within a given year) are reused from year to year. To overcome this fact we appended the year number to the consecutive number to ensure that all accidents are unique when we join by year.

- We excluded table from 2015 as it was missing some crucial columns.

- We noticed that there was an Emergency Arrival at Scene Column, along with time of accident. And therefore to find the Emergency response time we found the difference between the two, accounting for different unit of measurement types (see time_to_scene below)




In [ ]:
%%bigquery --project $project_id
CREATE OR REPLACE TABLE traffic_fatalities.traffic_features AS
SELECT
  CONCAT(accident2015.consecutive_number, accident2015.year_of_crash) AS id,    
  accident2015.state_number,
  accident2015.state_name,
  accident2015.number_of_motor_vehicles_in_transport_mvit,
  accident2015.number_of_parked_working_vehicles,
  accident2015.number_of_persons_in_motor_vehicles_in_transport_mvit,
  accident2015.number_of_persons_not_in_motor_vehicles_in_transport_mvit,
  accident2015.day_name,
  accident2015.month_of_crash_name,
  accident2015.day_of_week_name,
  accident2015.hour_of_crash_name,
  accident2015.land_use_name,
  accident2015.light_condition_name,
  accident2015.atmospheric_conditions_name,
  IF(accident2015.hour_of_notification > 23 OR accident2015.minute_of_notification > 59 OR accident2015.hour_of_arrival_at_scene > 23 OR accident2015.minute_of_arrival_at_scene > 59, 9999, (IF(accident2015.hour_of_notification <= accident2015.hour_of_arrival_at_scene, accident2015.hour_of_arrival_at_scene - accident2015.hour_of_notification, accident2015.hour_of_arrival_at_scene - accident2015.hour_of_notification + 24) * 60 + IF(accident2015.minute_of_notification <= accident2015.minute_of_arrival_at_scene, accident2015.minute_of_arrival_at_scene - accident2015.minute_of_notification, accident2015.minute_of_arrival_at_scene - accident2015.minute_of_notification + 60))) AS time_to_scene,
  IF(accident2015.hour_of_notification > 23 OR accident2015.minute_of_notification > 59 OR accident2015.hour_of_ems_arrival_at_hospital > 23 OR accident2015.minute_of_ems_arrival_at_hospital > 59, 9999, (IF(accident2015.hour_of_notification <= accident2015.hour_of_ems_arrival_at_hospital, accident2015.hour_of_ems_arrival_at_hospital - accident2015.hour_of_notification, accident2015.hour_of_ems_arrival_at_hospital - accident2015.hour_of_notification + 24) * 60 + IF(accident2015.minute_of_notification <= accident2015.minute_of_ems_arrival_at_hospital, accident2015.minute_of_ems_arrival_at_hospital - accident2015.minute_of_notification, accident2015.minute_of_ems_arrival_at_hospital - accident2015.minute_of_notification + 60))) AS time_to_hospital,
  accident2015.number_of_drunk_drivers,
  accident2015.number_of_fatalities,
  accident2015.number_of_fatalities AS label
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2015` AS accident2015
UNION ALL
-- (Your next query or continuation of the script)
SELECT
  CONCAT(accident2016.consecutive_number, accident2016.year_of_crash) AS id,    
  accident2016.state_number,
  accident2016.state_name,
  accident2016.number_of_motor_vehicles_in_transport_mvit,
  accident2016.number_of_parked_working_vehicles,
  accident2016.number_of_persons_in_motor_vehicles_in_transport_mvit,
  accident2016.number_of_persons_not_in_motor_vehicles_in_transport_mvit,
  accident2016.day_name,
  accident2016.month_of_crash_name,
  accident2016.day_of_week_name,
  accident2016.hour_of_crash_name,
  accident2016.land_use_name,
  accident2016.light_condition_name,
  accident2016.atmospheric_conditions_name,
  IF(accident2016.hour_of_notification > 23 OR accident2016.minute_of_notification > 59 OR accident2016.hour_of_arrival_at_scene > 23 OR accident2016.minute_of_arrival_at_scene > 59, 9999, (IF(accident2016.hour_of_notification <= accident2016.hour_of_arrival_at_scene, accident2016.hour_of_arrival_at_scene - accident2016.hour_of_notification, accident2016.hour_of_arrival_at_scene - accident2016.hour_of_notification + 24) * 60 + IF(accident2016.minute_of_notification <= accident2016.minute_of_arrival_at_scene, accident2016.minute_of_arrival_at_scene - accident2016.minute_of_notification, accident2016.minute_of_arrival_at_scene - accident2016.minute_of_notification + 60))) AS time_to_scene,
  IF(accident2016.hour_of_notification > 23 OR accident2016.minute_of_notification > 59 OR accident2016.hour_of_ems_arrival_at_hospital > 23 OR accident2016.minute_of_ems_arrival_at_hospital > 59, 9999, (IF(accident2016.hour_of_notification <= accident2016.hour_of_ems_arrival_at_hospital, accident2016.hour_of_ems_arrival_at_hospital - accident2016.hour_of_notification, accident2016.hour_of_ems_arrival_at_hospital - accident2016.hour_of_notification + 24) * 60 + IF(accident2016.minute_of_notification <= accident2016.minute_of_ems_arrival_at_hospital, accident2016.minute_of_ems_arrival_at_hospital - accident2016.minute_of_notification, accident2016.minute_of_ems_arrival_at_hospital - accident2016.minute_of_notification + 60))) AS time_to_hospital,
  accident2016.number_of_drunk_drivers,
  accident2016.number_of_fatalities,
  accident2016.number_of_fatalities AS label
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2016` AS accident2016
UNION ALL
SELECT
  CONCAT(accident2017.consecutive_number, accident2017.year_of_crash) AS id,    
  accident2017.state_number,
  accident2017.state_name,
  accident2017.number_of_motor_vehicles_in_transport_mvit,
  accident2017.number_of_parked_working_vehicles,
  accident2017.number_of_persons_in_motor_vehicles_in_transport_mvit,
  accident2017.number_of_persons_not_in_motor_vehicles_in_transport_mvit,
  accident2017.day_name,
  accident2017.month_of_crash_name,
  accident2017.day_of_week_name,
  accident2017.hour_of_crash_name,
  accident2017.land_use_name,
  accident2017.light_condition_name,
  accident2017.atmospheric_conditions_name,
  IF(accident2017.hour_of_notification > 23 OR accident2017.minute_of_notification > 59 OR accident2017.hour_of_arrival_at_scene > 23 OR accident2017.minute_of_arrival_at_scene > 59, 9999, (IF(accident2017.hour_of_notification <= accident2017.hour_of_arrival_at_scene, accident2017.hour_of_arrival_at_scene - accident2017.hour_of_notification, accident2017.hour_of_arrival_at_scene - accident2017.hour_of_notification + 24) * 60 + IF(accident2017.minute_of_notification <= accident2017.minute_of_arrival_at_scene, accident2017.minute_of_arrival_at_scene - accident2017.minute_of_notification, accident2017.minute_of_arrival_at_scene - accident2017.minute_of_notification + 60))) AS time_to_scene,
  IF(accident2017.hour_of_notification > 23 OR accident2017.minute_of_notification > 59 OR accident2017.hour_of_ems_arrival_at_hospital > 23 OR accident2017.minute_of_ems_arrival_at_hospital > 59, 9999, (IF(accident2017.hour_of_notification <= accident2017.hour_of_ems_arrival_at_hospital, accident2017.hour_of_ems_arrival_at_hospital - accident2017.hour_of_notification, accident2017.hour_of_ems_arrival_at_hospital - accident2017.hour_of_notification + 24) * 60 + IF(accident2017.minute_of_notification <= accident2017.minute_of_ems_arrival_at_hospital, accident2017.minute_of_ems_arrival_at_hospital - accident2017.minute_of_notification, accident2017.minute_of_ems_arrival_at_hospital - accident2017.minute_of_notification + 60))) AS time_to_hospital,
  accident2017.number_of_drunk_drivers,
  accident2017.number_of_fatalities,
  accident2017.number_of_fatalities AS label
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2017` AS accident2017
UNION ALL
SELECT
  CONCAT(accident2018.consecutive_number, accident2018.year_of_crash) AS id,    
  accident2018.state_number,
  accident2018.state_name,
  accident2018.number_of_motor_vehicles_in_transport_mvit,
  accident2018.number_of_parked_working_vehicles,
  accident2018.number_of_persons_in_motor_vehicles_in_transport_mvit,
  accident2018.number_of_persons_not_in_motor_vehicles_in_transport_mvit,
  accident2018.day_name,
  accident2018.month_of_crash_name,
  accident2018.day_of_week_name,
  accident2018.hour_of_crash_name,
  accident2018.land_use_name,
  accident2018.light_condition_name,
  accident2018.atmospheric_conditions_name,
  IF(accident2018.hour_of_notification > 23 OR accident2018.minute_of_notification > 59 OR accident2018.hour_of_arrival_at_scene > 23 OR accident2018.minute_of_arrival_at_scene > 59, 9999, (IF(accident2018.hour_of_notification <= accident2018.hour_of_arrival_at_scene, accident2018.hour_of_arrival_at_scene - accident2018.hour_of_notification, accident2018.hour_of_arrival_at_scene - accident2018.hour_of_notification + 24) * 60 + IF(accident2018.minute_of_notification <= accident2018.minute_of_arrival_at_scene, accident2018.minute_of_arrival_at_scene - accident2018.minute_of_notification, accident2018.minute_of_arrival_at_scene - accident2018.minute_of_notification + 60))) AS time_to_scene,
  IF(accident2018.hour_of_notification > 23 OR accident2018.minute_of_notification > 59 OR accident2018.hour_of_ems_arrival_at_hospital > 23 OR accident2018.minute_of_ems_arrival_at_hospital > 59, 9999, (IF(accident2018.hour_of_notification <= accident2018.hour_of_ems_arrival_at_hospital, accident2018.hour_of_ems_arrival_at_hospital - accident2018.hour_of_notification, accident2018.hour_of_ems_arrival_at_hospital - accident2018.hour_of_notification + 24) * 60 + IF(accident2018.minute_of_notification <= accident2018.minute_of_ems_arrival_at_hospital, accident2018.minute_of_ems_arrival_at_hospital - accident2018.minute_of_notification, accident2018.minute_of_ems_arrival_at_hospital - accident2018.minute_of_notification + 60))) AS time_to_hospital,
  accident2018.number_of_drunk_drivers,
  accident2018.number_of_fatalities,
  accident2018.number_of_fatalities AS label
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2018` AS accident2018
UNION ALL
SELECT
  CONCAT(accident2019.consecutive_number, accident2019.year_of_crash) AS id,    
  accident2019.state_number,
  accident2019.state_name,
  accident2019.number_of_motor_vehicles_in_transport_mvit,
  accident2019.number_of_parked_working_vehicles,
  accident2019.number_of_persons_in_motor_vehicles_in_transport_mvit,
  accident2019.number_of_persons_not_in_motor_vehicles_in_transport_mvit,
  accident2019.day_name,
  accident2019.month_of_crash_name,
  accident2019.day_of_week_name,
  accident2019.hour_of_crash_name,
  accident2019.land_use_name,
  accident2019.light_condition_name,
  accident2019.atmospheric_conditions_name,
  IF(accident2019.hour_of_notification > 23 OR accident2019.minute_of_notification > 59 OR accident2019.hour_of_arrival_at_scene > 23 OR accident2019.minute_of_arrival_at_scene > 59, 9999, (IF(accident2019.hour_of_notification <= accident2019.hour_of_arrival_at_scene, accident2019.hour_of_arrival_at_scene - accident2019.hour_of_notification, accident2019.hour_of_arrival_at_scene - accident2019.hour_of_notification + 24) * 60 + IF(accident2019.minute_of_notification <= accident2019.minute_of_arrival_at_scene, accident2019.minute_of_arrival_at_scene - accident2019.minute_of_notification, accident2019.minute_of_arrival_at_scene - accident2019.minute_of_notification + 60))) AS time_to_scene,
  IF(accident2019.hour_of_notification > 23 OR accident2019.minute_of_notification > 59 OR accident2019.hour_of_ems_arrival_at_hospital > 23 OR accident2019.minute_of_ems_arrival_at_hospital > 59, 9999, (IF(accident2019.hour_of_notification <= accident2019.hour_of_ems_arrival_at_hospital, accident2019.hour_of_ems_arrival_at_hospital - accident2019.hour_of_notification, accident2019.hour_of_ems_arrival_at_hospital - accident2019.hour_of_notification + 24) * 60 + IF(accident2019.minute_of_notification <= accident2019.minute_of_ems_arrival_at_hospital, accident2019.minute_of_ems_arrival_at_hospital - accident2019.minute_of_notification, accident2019.minute_of_ems_arrival_at_hospital - accident2019.minute_of_notification + 60))) AS time_to_hospital,
  accident2019.number_of_drunk_drivers,
  accident2019.number_of_fatalities,
  accident2019.number_of_fatalities AS label
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2019` AS accident2019
UNION ALL
SELECT
  CONCAT(accident2020.consecutive_number, accident2020.year_of_crash) AS id,    
  accident2020.state_number,
  accident2020.state_name,
  accident2020.number_of_motor_vehicles_in_transport_mvit,
  accident2020.number_of_parked_working_vehicles,
  accident2020.number_of_persons_in_motor_vehicles_in_transport_mvit,
  accident2020.number_of_persons_not_in_motor_vehicles_in_transport_mvit,
  accident2020.day_name,
  accident2020.month_of_crash_name,
  accident2020.day_of_week_name,
  accident2020.hour_of_crash_name,
  accident2020.land_use_name,
  accident2020.light_condition_name,
  accident2020.atmospheric_conditions_1_name AS atmospheric_conditions_name,
  IF(accident2020.hour_of_notification > 23 OR accident2020.minute_of_notification > 59 OR accident2020.hour_of_arrival_at_scene > 23 OR accident2020.minute_of_arrival_at_scene > 59, 9999, (IF(accident2020.hour_of_notification <= accident2020.hour_of_arrival_at_scene, accident2020.hour_of_arrival_at_scene - accident2020.hour_of_notification, accident2020.hour_of_arrival_at_scene - accident2020.hour_of_notification + 24) * 60 + IF(accident2020.minute_of_notification <= accident2020.minute_of_arrival_at_scene, accident2020.minute_of_arrival_at_scene - accident2020.minute_of_notification, accident2020.minute_of_arrival_at_scene - accident2020.minute_of_notification + 60))) AS time_to_scene,
  IF(accident2020.hour_of_notification > 23 OR accident2020.minute_of_notification > 59 OR accident2020.hour_of_ems_arrival_at_hospital > 23 OR accident2020.minute_of_ems_arrival_at_hospital > 59, 9999, (IF(accident2020.hour_of_notification <= accident2020.hour_of_ems_arrival_at_hospital, accident2020.hour_of_ems_arrival_at_hospital - accident2020.hour_of_notification, accident2020.hour_of_ems_arrival_at_hospital - accident2020.hour_of_notification + 24) * 60 + IF(accident2020.minute_of_notification <= accident2020.minute_of_ems_arrival_at_hospital, accident2020.minute_of_ems_arrival_at_hospital - accident2020.minute_of_notification, accident2020.minute_of_ems_arrival_at_hospital - accident2020.minute_of_notification + 60))) AS time_to_hospital,
  accident2020.number_of_drunk_drivers,
  accident2020.number_of_fatalities,
  accident2020.number_of_fatalities AS label
FROM `bigquery-public-data.nhtsa_traffic_fatalities. accident_2020` AS accident2020

### Total Number of Accidents

Here we see that there are ~200,000 fatal crashes in the US from 2015-2020. This number is quite daunting, and a little higher than what we were expecting. Perhaps a shift to public transport won't just mitigate the environmental damage caused by cars - it will save many lives too...

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)
FROM `traffic_fatalities.traffic_features`

### Total Number of Fatalitites

There have been 221137 fatalities in the past 6 years as a result of car accidents. Therefore we can infer that most fatal car accidents only result in one death. However there are still many accidents which involve multiple fatalities. 

In [ ]:
%%bigquery --project $project_id
SELECT SUM(number_of_fatalities)
FROM `traffic_fatalities.traffic_features`

### Preliminary State Investigations

As we investigate state differences, it is very important to account of the different populations. Some states may have more fatal crashes simply due to having a bigger population. Just so, we will join on 5 year census data from 2020, and we join on state_number. No surprises that California, Texas and Florida have the biggest populations in the US.

In [ ]:
%%bigquery --project $project_id

SELECT t.state_name, s.total_pop
FROM `traffic_fatalities.traffic_features` AS t
JOIN `bigquery-public-data.census_bureau_acs.state_2020_5yr` AS s ON t.state_number = CAST(s.geo_id AS INT64)
GROUP BY state_name, total_pop
ORDER BY total_pop DESC

ACCIDENTS PER STATE
- The below query shows the number of fatal accidents 2015-2020 by state. Comparing to the above table we can see that the number of fatal accidents are highly correlated with the population of the given state.

In [ ]:
%%bigquery --project $project_id

WITH state_accidents AS (
    SELECT COUNT(*) AS num_accidents, state_name, state_number
    FROM `traffic_fatalities.traffic_features`
    GROUP BY state_name, state_number
)
SELECT f.num_accidents, f.state_name, s.state
FROM state_accidents AS f
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name
ORDER BY num_accidents DESC

In [ ]:
query_total_accidents_per_state = "WITH state_accidents AS (SELECT COUNT(*) AS num_accidents, state_name, state_number FROM `traffic_fatalities.traffic_features` GROUP BY state_name, state_number) SELECT f.num_accidents AS total_accidents_per_state, f.state_name, s.state FROM state_accidents AS f JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name"
total_accidents_per_state = pd.read_gbq(query_total_accidents_per_state, project_id=project_id, dialect='standard')
fig_total_accidents_per_state = px.choropleth(
    total_accidents_per_state,
    locations='state',
    locationmode='USA-states',
    color='total_accidents_per_state',
    color_continuous_scale = 'Reds',
    title='Total Accidents by State',
)

fig_total_accidents_per_state.update_layout(
    title_text = 'Accidents per State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

ACCIDENTS PER CAPITA

- Because of correlation between # accidents and state population, we must control for the differences in Population, and find which states have the highest amount of fatal accidents per person. This was done below and we now see that Mississippi has the highest incidence of fatal accidents per person in a year. Similarly, Wyoming, South Carolina, Alabama and Montana are the following states with highest amount of fatal accidents per capita. This makes us wonder whether or not this is because of higher rates of drunk driving, unfavourable atmospheric conditions, higher use of highways or just poor road education. We will investigate this further later in the project...

In [ ]:
%%bigquery --project $project_id

WITH state_accidents AS (
    SELECT COUNT(*) AS num_accidents, state_name, state_number
    FROM `traffic_fatalities.traffic_features`
    GROUP BY state_name, state_number
)

SELECT f.num_accidents / p.total_pop / 6 * 100 AS accidents_per_capita_per_year, f.state_name, s.state
FROM state_accidents AS f
JOIN `bigquery-public-data.census_bureau_acs.state_2020_5yr` AS p ON f.state_number = CAST(p.geo_id AS INT64)
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name
ORDER BY percent_accidents_per_year DESC

In [ ]:
query_percent_accidents_per_state = "WITH state_accidents AS (SELECT COUNT(*) AS num_accidents, state_name, state_number FROM `traffic_fatalities.traffic_features` GROUP BY state_name, state_number) SELECT f.num_accidents / p.total_pop / 6 * 100 AS percent_accidents_per_year, f.state_name, s.state FROM state_accidents AS f JOIN `bigquery-public-data.census_bureau_acs.state_2020_5yr` AS p ON f.state_number = CAST(p.geo_id AS INT64) JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name"
percent_accidents_per_state = pd.read_gbq(query_percent_accidents_per_state, project_id=project_id, dialect='standard')
fig_percent_accidents_per_state = px.choropleth(
    percent_accidents_per_state,
    locations='state',
    locationmode='USA-states',
    color='accidents_per_capita_per_year',
    color_continuous_scale = 'Reds',
    title='Accidents Per Capita by Year',
)

fig_percent_accidents_per_state.update_layout(
    title_text = 'Accidents per Person Per Year by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id

WITH state_fatalities AS (
    SELECT SUM(number_of_fatalities) AS num_fatalities, state_name, state_number
    FROM `traffic_fatalities.traffic_features`
    GROUP BY state_name, state_number
)
SELECT f.num_fatalities, f.state_name, s.state
FROM state_fatalities AS f
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name
ORDER BY num_fatalities DESC

FATALITIES BY STATE
- Below we conduct a similar analysis for fatalities per state 
- Here we see like accidents, it is highly correlated with population of the given state
- Hence when controlling for population we expect to see that the ranking of states for traffic fatalities per capita should mirror fatal accidents per capita as well

In [ ]:
query_fatalities_per_state = "WITH state_fatalities AS (SELECT SUM(number_of_fatalities) AS num_fatalities, state_name, state_number FROM `traffic_fatalities.traffic_features` GROUP BY state_name, state_number) SELECT f.num_fatalities AS total_fatalities, f.state_name, s.state FROM state_fatalities AS f JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name"
fatalities_per_state = pd.read_gbq(query_fatalities_per_state, project_id=project_id, dialect='standard')
fig_total_fatalities_per_state = px.choropleth(
    fatalities_per_state,
    locations='state',
    locationmode='USA-states',
    color='total_fatalities',
    color_continuous_scale = 'Reds',
    title='Total Fatalities by State',
)

fig_total_fatalities_per_state.update_layout(
    title_text = 'Total Fatalities by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id

WITH state_fatalities AS (
    SELECT SUM(number_of_fatalities) AS num_fatalities, state_name, state_number
    FROM `traffic_fatalities.traffic_features`
    GROUP BY state_name, state_number
)

SELECT f.num_fatalities / p.total_pop / 6 * 100 AS fatalities_per_capita_per_year, f.state_name, s.state
FROM state_fatalities AS f
JOIN `bigquery-public-data.census_bureau_acs.state_2020_5yr` AS p ON f.state_number = CAST(p.geo_id AS INT64)
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name
ORDER BY percent_fatalities_per_year DESC

As expected we see that the number of fatalities per capita per year is mostly the same as fatal accidents per capita per years in terms of state rankings.

In [ ]:
query_percent_fatalities_per_state = "WITH state_fatalities AS (SELECT SUM(number_of_fatalities) AS num_fatalities, state_name, state_number FROM `traffic_fatalities.traffic_features`GROUP BY state_name, state_number) SELECT f.num_fatalities / p.total_pop / 6 * 100 AS percent_fatalities_per_year, f.state_name, s.state FROM state_fatalities AS f JOIN `bigquery-public-data.census_bureau_acs.state_2020_5yr` AS p ON f.state_number = CAST(p.geo_id AS INT64) JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name"
percent_fatalities_per_state = pd.read_gbq(query_percent_fatalities_per_state, project_id=project_id, dialect='standard')
fig_percent_fatalities_per_state = px.choropleth(
    percent_fatalities_per_state,
    locations='state',
    locationmode='USA-states',
    color='fatalities_per_capita_per_year',
    color_continuous_scale = 'Reds',
    title='Fatalities Per Capita Per Year by State',
)

fig_percent_fatalities_per_state.update_layout(
    title_text = 'Percent Fatalities per Year by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

AVERAGE FATALITIES PER ACCIDENT 
- Now we want to find if there is a difference between number of fatalities per accident by state 

- Interestingly we see that the data is vastly different to the results above. Here we see that the states with the deadliest fatal accidents on average are Wyoming, South Dakota, Nebraska, Kansas and New Mexico. We postulate that this may be due to combination of factors like the fact that incidence of Ice and snow may be higher in these states along with the level of alcohol comsumption. Moreover indirect factors such as level of education in these states may increase incidences of reckless driving etc.
- We see increased fatalities per fatal crash on average as states become more landlocked. We wonder if this is because of higher temperature variation for states further away from the coastline which may increase incidence of unfavourable weather conditions for driving in these states. Moreover one could also argue that the level of education may be higher along the coastlines and in states with higher population (with Texas being an outlier here) which could have a correlation with amount of reckless driving and incidence of high severity fatal crashes. In addition, as aforementioned these states may also occur in states with higher levels of drunk driving.



In [ ]:
%%bigquery --project $project_id

WITH state_fatalities_accidents AS (
    SELECT AVG(number_of_fatalities) AS avg_fatalities, state_name, state_number
    FROM `traffic_fatalities.traffic_features`
    GROUP BY state_name, state_number
)

SELECT f.avg_fatalities AS average_fatalities, f.state_name, s.state
FROM state_fatalities_accidents AS f
JOIN `bigquery-public-data.census_bureau_acs.state_2020_5yr` AS p ON f.state_number = CAST(p.geo_id AS INT64)
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name
ORDER BY average_fatalities DESC

In [ ]:
query_fatalities_accidents_per_state = "WITH state_fatalities_accidents AS (SELECT AVG(number_of_fatalities) AS avg_fatalities, state_name, state_number FROM `traffic_fatalities.traffic_features` GROUP BY state_name, state_number) SELECT f.avg_fatalities, f.state_name, s.state FROM state_fatalities_accidents AS f JOIN `bigquery-public-data.census_bureau_acs.state_2020_5yr` AS p ON f.state_number = CAST(p.geo_id AS INT64) JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON f.state_name = s.state_name"
fatalities_accidents_per_state = pd.read_gbq(query_fatalities_accidents_per_state, project_id=project_id, dialect='standard')
fig_fatalities_accidents_per_state = px.choropleth(
    fatalities_accidents_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Reds',
    title='Average Fatalities by State',
)

fig_fatalities_accidents_per_state.update_layout(
    title_text = 'Average Fatalities by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

### Day of Week

- Much of our prediction here is related to drunk driving. We believe that Fridays and Saturdays will have the highest amount of fatal crashes due to this
- Interestingly Saturday and Sunday have the highest incidence of fatal crashes as opposed to Fridays and Saturday. Upon further thought this makes sense as we predict that many crashes that occur on Sunday happen between 12-4am and are alcohol related. Moreover, the spillover of partygoers on Friday into the early hours of Saturday morning bumps up the numbers for Saturday
- Thursday is the next highest day outside of "the weekend" in terms of fatal accidents. We don't know why this is, perhaps this could be due to work fatigue etc. We wonder if Thursday has the fourth highest rate of accidents that involve drunk drivers?

In [ ]:
%%bigquery --project $project_id

SELECT COUNT(*) as num_crashes, day_of_week_name
FROM `traffic_fatalities.traffic_features`
GROUP BY day_of_week_name
ORDER BY num_crashes DESC

In [ ]:
query_days_accidents  = "SELECT COUNT(*) as num_crashes, day_of_week_name FROM `traffic_fatalities.traffic_features` GROUP BY day_of_week_name ORDER BY CASE WHEN day_of_week_name = 'Moday' THEN 0 WHEN day_of_week_name = 'Tuesday' THEN 1 WHEN day_of_week_name = 'Wednesday' THEN 3 WHEN day_of_week_name = 'Thursday' THEN 4 WHEN day_of_week_name = 'Friday' THEN 5 WHEN day_of_week_name = 'Saturday' THEN 6 WHEN day_of_week_name = 'Sunday' THEN 7 END"

days_accidents = pd.read_gbq(query_days_accidents, project_id=project_id, dialect='standard')

plt.bar(days_accidents["day_of_week_name"], days_accidents["num_crashes"])

- As expected most states have either Saturdays, Sundays or Fridays as their deadliest days  

In [ ]:
%%bigquery --project $project_id

SELECT SUM(number_of_fatalities) as num_fatalities, day_of_week_name
FROM `traffic_fatalities.traffic_features`
GROUP BY day_of_week_name
ORDER BY num_fatalities DESC

In [ ]:
query_days_fatalities  = "SELECT SUM(number_of_fatalities) as num_fatalities, day_of_week_name FROM `traffic_fatalities.traffic_features` GROUP BY day_of_week_name ORDER BY CASE WHEN day_of_week_name = 'Moday' THEN 0 WHEN day_of_week_name = 'Tuesday' THEN 1 WHEN day_of_week_name = 'Wednesday' THEN 3 WHEN day_of_week_name = 'Thursday' THEN 4 WHEN day_of_week_name = 'Friday' THEN 5 WHEN day_of_week_name = 'Saturday' THEN 6 WHEN day_of_week_name = 'Sunday' THEN 7 END"

days_fatalities = pd.read_gbq(query_days_fatalities, project_id=project_id, dialect='standard')

plt.bar(days_fatalities["day_of_week_name"], days_fatalities["num_fatalities"])

In [ ]:
%%bigquery --project $project_id

WITH state_weekdays_fatalities AS (
    SELECT SUM(number_of_fatalities) AS num_fatalities, state_name, day_of_week_name, RANK() OVER (PARTITION BY state_name ORDER BY SUM(number_of_fatalities) DESC) AS rank
    FROM `traffic_fatalities.traffic_features`
    GROUP BY state_name, state_number, day_of_week_name
    ORDER BY rank
)
SELECT w.num_fatalities/t.total_fatalities_per_state AS percent_fatalities_per_week, w.day_of_week_name, w.state_name, s.state
FROM state_weekdays_fatalities AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name
JOIN (SELECT state_name, SUM(number_of_fatalities) AS total_fatalities_per_state FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE rank = 1
ORDER BY state_name

In [ ]:
query_weekday_fatalities_state = "WITH state_weekdays_fatalities AS (SELECT SUM(number_of_fatalities) AS num_fatalities, state_name, day_of_week_name, RANK() OVER (PARTITION BY state_name ORDER BY SUM(number_of_fatalities) DESC) AS rank FROM `traffic_fatalities.traffic_features` GROUP BY state_name, state_number, day_of_week_name ORDER BY rank) SELECT w.num_fatalities, w.day_of_week_name, w.state_name, s.state FROM state_weekdays_fatalities AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name WHERE rank = 1 ORDER BY CASE WHEN day_of_week_name = 'Moday' THEN 0 WHEN day_of_week_name = 'Tuesday' THEN 1 WHEN day_of_week_name = 'Wednesday' THEN 3 WHEN day_of_week_name = 'Thursday' THEN 4 WHEN day_of_week_name = 'Friday' THEN 5 WHEN day_of_week_name = 'Saturday' THEN 6 WHEN day_of_week_name = 'Sunday' THEN 7 END"
weekday_fatalities_state = pd.read_gbq(query_weekday_fatalities_state, project_id=project_id, dialect='standard')
fig_weekday_fatalities_state = px.choropleth(
    weekday_fatalities_state,
    locations='state',
    locationmode='USA-states',
    color='day_of_week_name',
    color_discrete_map = days_of_week_color_dict,
    title='Days of Week with Most fatalitites',
)

fig_weekday_fatalities_state.update_layout(
    title_text = 'Days of Week with Most fatalitites by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id

SELECT AVG(number_of_fatalities) as avg_fatalities, day_of_week_name
FROM `traffic_fatalities.traffic_features`
GROUP BY day_of_week_name
ORDER BY avg_fatalities DESC

In [ ]:
query_days_fatalities_accidents  = "SELECT AVG(number_of_fatalities) as avg_fatalities, day_of_week_name FROM `traffic_fatalities.traffic_features` GROUP BY day_of_week_name ORDER BY CASE WHEN day_of_week_name = 'Moday' THEN 0 WHEN day_of_week_name = 'Tuesday' THEN 1 WHEN day_of_week_name = 'Wednesday' THEN 3 WHEN day_of_week_name = 'Thursday' THEN 4 WHEN day_of_week_name = 'Friday' THEN 5 WHEN day_of_week_name = 'Saturday' THEN 6 WHEN day_of_week_name = 'Sunday' THEN 7 END"

days_fatalities_accidents = pd.read_gbq(query_days_fatalities_accidents, project_id=project_id, dialect='standard')

plt.bar(days_fatalities_accidents["day_of_week_name"], days_fatalities_accidents["avg_fatalities"])

In [ ]:
%%bigquery --project $project_id

WITH state_weekdays_fatalities_accidents AS (
    SELECT AVG(number_of_fatalities) AS avg_fatalities, state_name, day_of_week_name, RANK() OVER (PARTITION BY state_name ORDER BY AVG(number_of_fatalities) DESC) AS rank
    FROM `traffic_fatalities.traffic_features`
    GROUP BY state_name, state_number, day_of_week_name
    ORDER BY rank
)
SELECT w.avg_fatalities, w.day_of_week_name, w.state_name, s.state
FROM state_weekdays_fatalities_accidents AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name
WHERE rank = 1
ORDER BY state_name

**REPLACE THIS WITH of counts of states for respective day with highest average fatalities per crash**
- To our surprise we found that there were several states that had wednesday as the day with highest average fatalities per accident. We wonder if this occurs due to differences in drinking habits in these states.

In [ ]:
query_weekday_fatalities_accidents_state = "WITH state_weekdays_fatalities_accidents AS (SELECT AVG(number_of_fatalities) AS avg_fatalities, state_name, day_of_week_name, RANK() OVER (PARTITION BY state_name ORDER BY AVG(number_of_fatalities) DESC) AS rank FROM `traffic_fatalities.traffic_features` GROUP BY state_name, state_number, day_of_week_name ORDER BY rank) SELECT w.avg_fatalities, w.day_of_week_name, w.state_name, s.state FROM state_weekdays_fatalities_accidents AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name WHERE rank = 1 ORDER BY CASE WHEN day_of_week_name = 'Moday' THEN 0 WHEN day_of_week_name = 'Tuesday' THEN 1 WHEN day_of_week_name = 'Wednesday' THEN 3 WHEN day_of_week_name = 'Thursday' THEN 4 WHEN day_of_week_name = 'Friday' THEN 5 WHEN day_of_week_name = 'Saturday' THEN 6 WHEN day_of_week_name = 'Sunday' THEN 7 END"
weekday_fatalities_accidents_state = pd.read_gbq(query_weekday_fatalities_accidents_state, project_id=project_id, dialect='standard')
fig_weekday_fatalities_accidents_state = px.choropleth(
    weekday_fatalities_accidents_state,
    locations='state',
    locationmode='USA-states',
    color='day_of_week_name',
    color_discrete_map = days_of_week_color_dict,
    title='Days of Week with Highest Fatalities to Accident Ratio',
)

fig_weekday_fatalities_accidents_state.update_layout(
    title_text = 'Days of Week with Highest Average Fatalities per Crash by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

### Hour
- We expect that the highest amount of fatal accidents occur at night, or during rush hour (8-10am, 5-7pm, and 9pm - 2am)
- We also expect numbers to increase for 9pm - 2am time period on the weekends due to increased drunkeness 
- The below results align with out hypothesis

In [ ]:
%%bigquery --project $project_id

SELECT COUNT(*) AS num_crashes, hour_of_crash_name 
FROM `traffic_fatalities.traffic_features` 
GROUP BY hour_of_crash_name
ORDER BY num_crashes DESC

In [ ]:
%%bigquery --project $project_id

SELECT COUNT(*) AS num_crashes, hour_of_crash_name, day_of_week_name 
FROM `traffic_fatalities.traffic_features` 
GROUP BY hour_of_crash_name, day_of_week_name
ORDER BY num_crashes DESC

In [ ]:
num_hours = range(24)

In [ ]:
# All Severities
query_hours_accidents  = "SELECT COUNT(*) as num_crashes, hour_of_crash_name FROM `traffic_fatalities.traffic_features` GROUP BY hour_of_crash_name ORDER BY CASE WHEN hour_of_crash_name = '0:00am-0:59am' THEN 0 WHEN hour_of_crash_name = '1:00am-1:59am' THEN 1 WHEN hour_of_crash_name = '2:00am-2:59am' THEN 2 WHEN hour_of_crash_name = '3:00am-3:59am' THEN 3 WHEN hour_of_crash_name = '4:00am-4:59am' THEN 4 WHEN hour_of_crash_name = '5:00am-5:59am' THEN 5 WHEN hour_of_crash_name = '6:00am-6:59am' THEN 6 WHEN hour_of_crash_name = '7:00am-7:59am' THEN 7 WHEN hour_of_crash_name = '8:00am-8:59am' THEN 8 WHEN hour_of_crash_name = '9:00am-9:59am' THEN 9 WHEN hour_of_crash_name = '10:00am-10:59am' THEN 10 WHEN hour_of_crash_name = '11:00am-11:59am' THEN 11 WHEN hour_of_crash_name = '12:00pm-12:59pm' THEN 12 WHEN hour_of_crash_name = '1:00pm-1:59pm' THEN 13 WHEN hour_of_crash_name = '2:00pm-2:59pm' THEN 14 WHEN hour_of_crash_name = '3:00pm-3:59pm' THEN 15 WHEN hour_of_crash_name = '4:00pm-4:59pm' THEN 16 WHEN hour_of_crash_name = '5:00pm-5:59pm' THEN 17 WHEN hour_of_crash_name = '6:00pm-6:59pm' THEN 18 WHEN hour_of_crash_name = '7:00pm-7:59pm' THEN 19 WHEN hour_of_crash_name = '8:00pm-8:59pm' THEN 20 WHEN hour_of_crash_name = '9:00pm-9:59pm' THEN 21 WHEN hour_of_crash_name = '10:00pm-10:59pm' THEN 22 WHEN hour_of_crash_name = '11:00pm-11:59pm' THEN 23 END"

hours_accidents = pd.read_gbq(query_hours_accidents, project_id=project_id, dialect='standard')

plt.bar(num_hours, hours_accidents["num_crashes"])

In [ ]:
%%bigquery --project $project_id

SELECT SUM(number_of_fatalities) AS num_fatalities, hour_of_crash_name 
FROM `traffic_fatalities.traffic_features` 
GROUP BY hour_of_crash_name
ORDER BY num_fatalities DESC

In [ ]:
query_hours_fatalities  = "SELECT SUM(number_of_fatalities) AS num_fatalities, hour_of_crash_name FROM `traffic_fatalities.traffic_features` GROUP BY hour_of_crash_name ORDER BY CASE WHEN hour_of_crash_name = '0:00am-0:59am' THEN 0 WHEN hour_of_crash_name = '1:00am-1:59am' THEN 1 WHEN hour_of_crash_name = '2:00am-2:59am' THEN 2 WHEN hour_of_crash_name = '3:00am-3:59am' THEN 3 WHEN hour_of_crash_name = '4:00am-4:59am' THEN 4 WHEN hour_of_crash_name = '5:00am-5:59am' THEN 5 WHEN hour_of_crash_name = '6:00am-6:59am' THEN 6 WHEN hour_of_crash_name = '7:00am-7:59am' THEN 7 WHEN hour_of_crash_name = '8:00am-8:59am' THEN 8 WHEN hour_of_crash_name = '9:00am-9:59am' THEN 9 WHEN hour_of_crash_name = '10:00am-10:59am' THEN 10 WHEN hour_of_crash_name = '11:00am-11:59am' THEN 11 WHEN hour_of_crash_name = '12:00pm-12:59pm' THEN 12 WHEN hour_of_crash_name = '1:00pm-1:59pm' THEN 13 WHEN hour_of_crash_name = '2:00pm-2:59pm' THEN 14 WHEN hour_of_crash_name = '3:00pm-3:59pm' THEN 15 WHEN hour_of_crash_name = '4:00pm-4:59pm' THEN 16 WHEN hour_of_crash_name = '5:00pm-5:59pm' THEN 17 WHEN hour_of_crash_name = '6:00pm-6:59pm' THEN 18 WHEN hour_of_crash_name = '7:00pm-7:59pm' THEN 19 WHEN hour_of_crash_name = '8:00pm-8:59pm' THEN 20 WHEN hour_of_crash_name = '9:00pm-9:59pm' THEN 21 WHEN hour_of_crash_name = '10:00pm-10:59pm' THEN 22 WHEN hour_of_crash_name = '11:00pm-11:59pm' THEN 23 END"

hours_fatalities = pd.read_gbq(query_hours_fatalities, project_id=project_id, dialect='standard')

plt.bar(num_hours, hours_fatalities["num_fatalities"])

In [ ]:
%%bigquery --project $project_id

SELECT AVG(number_of_fatalities) AS avg_fatalities, hour_of_crash_name 
FROM `traffic_fatalities.traffic_features` 
GROUP BY hour_of_crash_name
ORDER BY avg_fatalities DESC

In [ ]:
query_hours_fatalities_accidents  = "SELECT AVG(number_of_fatalities) AS avg_fatalities, hour_of_crash_name FROM `traffic_fatalities.traffic_features` GROUP BY hour_of_crash_name ORDER BY CASE WHEN hour_of_crash_name = '0:00am-0:59am' THEN 0 WHEN hour_of_crash_name = '1:00am-1:59am' THEN 1 WHEN hour_of_crash_name = '2:00am-2:59am' THEN 2 WHEN hour_of_crash_name = '3:00am-3:59am' THEN 3 WHEN hour_of_crash_name = '4:00am-4:59am' THEN 4 WHEN hour_of_crash_name = '5:00am-5:59am' THEN 5 WHEN hour_of_crash_name = '6:00am-6:59am' THEN 6 WHEN hour_of_crash_name = '7:00am-7:59am' THEN 7 WHEN hour_of_crash_name = '8:00am-8:59am' THEN 8 WHEN hour_of_crash_name = '9:00am-9:59am' THEN 9 WHEN hour_of_crash_name = '10:00am-10:59am' THEN 10 WHEN hour_of_crash_name = '11:00am-11:59am' THEN 11 WHEN hour_of_crash_name = '12:00pm-12:59pm' THEN 12 WHEN hour_of_crash_name = '1:00pm-1:59pm' THEN 13 WHEN hour_of_crash_name = '2:00pm-2:59pm' THEN 14 WHEN hour_of_crash_name = '3:00pm-3:59pm' THEN 15 WHEN hour_of_crash_name = '4:00pm-4:59pm' THEN 16 WHEN hour_of_crash_name = '5:00pm-5:59pm' THEN 17 WHEN hour_of_crash_name = '6:00pm-6:59pm' THEN 18 WHEN hour_of_crash_name = '7:00pm-7:59pm' THEN 19 WHEN hour_of_crash_name = '8:00pm-8:59pm' THEN 20 WHEN hour_of_crash_name = '9:00pm-9:59pm' THEN 21 WHEN hour_of_crash_name = '10:00pm-10:59pm' THEN 22 WHEN hour_of_crash_name = '11:00pm-11:59pm' THEN 23 END"

hours_fatalities_accidents = pd.read_gbq(query_hours_fatalities_accidents, project_id=project_id, dialect='standard')

plt.bar(num_hours, hours_fatalities_accidents["avg_fatalities"])

In [ ]:
%%bigquery --project $project_id

WITH state_hours_accidents AS (
    SELECT COUNT(*) AS num_crashes, state_name, hour_of_crash_name, RANK() OVER (PARTITION BY state_name ORDER BY COUNT(*) DESC) AS rank 
    FROM `traffic_fatalities.traffic_features` 
    GROUP BY state_name, state_number, hour_of_crash_name 
    ORDER BY rank
    ) 
SELECT w.num_crashes / t.total_accidents_per_state AS percent_accidents_per_day, w.hour_of_crash_name, w.state_name, s.state 
FROM state_hours_accidents AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT state_name, COUNT(*) AS total_accidents_per_state FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE rank = 1
ORDER BY state_name

### Month

In [ ]:
%%bigquery --project $project_id

SELECT COUNT(*) as num_crashes, month_of_crash_name
FROM `traffic_fatalities.traffic_features`
GROUP BY month_of_crash_name
ORDER BY num_crashes DESC

In [ ]:
num_months = range(1, 13)

In [ ]:
query_months_accidents  = "SELECT COUNT(*) as num_crashes, month_of_crash_name FROM `traffic_fatalities.traffic_features` GROUP BY month_of_crash_name ORDER BY CASE WHEN month_of_crash_name = 'January' THEN 1 WHEN month_of_crash_name = 'February' THEN 2 WHEN month_of_crash_name = 'March' THEN 3 WHEN month_of_crash_name = 'April' THEN 4 WHEN month_of_crash_name = 'May' THEN 5 WHEN month_of_crash_name = 'June' THEN 6 WHEN month_of_crash_name = 'July' THEN 7 WHEN month_of_crash_name = 'August' THEN 8 WHEN month_of_crash_name = 'September' THEN 9 WHEN month_of_crash_name = 'October' THEN 10 WHEN month_of_crash_name = 'November' THEN 11 WHEN month_of_crash_name = 'December' THEN 12 END"

months_accidents = pd.read_gbq(query_months_accidents, project_id=project_id, dialect='standard')

plt.bar(num_months, months_accidents["num_crashes"])

In [ ]:
%%bigquery --project $project_id

SELECT SUM(number_of_fatalities) as num_fatalities, month_of_crash_name
FROM `traffic_fatalities.traffic_features`
GROUP BY month_of_crash_name
ORDER BY num_fatalities DESC

In [ ]:
query_months_fatalities  = "SELECT SUM(number_of_fatalities) as num_fatalities, month_of_crash_name FROM `traffic_fatalities.traffic_features` GROUP BY month_of_crash_name ORDER BY CASE WHEN month_of_crash_name = 'January' THEN 1 WHEN month_of_crash_name = 'February' THEN 2 WHEN month_of_crash_name = 'March' THEN 3 WHEN month_of_crash_name = 'April' THEN 4 WHEN month_of_crash_name = 'May' THEN 5 WHEN month_of_crash_name = 'June' THEN 6 WHEN month_of_crash_name = 'July' THEN 7 WHEN month_of_crash_name = 'August' THEN 8 WHEN month_of_crash_name = 'September' THEN 9 WHEN month_of_crash_name = 'October' THEN 10 WHEN month_of_crash_name = 'November' THEN 11 WHEN month_of_crash_name = 'December' THEN 12 END"

months_fatalities = pd.read_gbq(query_months_fatalities, project_id=project_id, dialect='standard')

plt.bar(num_months, months_fatalities["num_fatalities"])

In [ ]:
%%bigquery --project $project_id

SELECT AVG(number_of_fatalities) as avg_fatalities, month_of_crash_name
FROM `traffic_fatalities.traffic_features`
GROUP BY month_of_crash_name
ORDER BY avg_fatalities DESC

In [ ]:
query_months_avg_fatalities  = "SELECT AVG(number_of_fatalities) as avg_fatalities, month_of_crash_name FROM `traffic_fatalities.traffic_features` GROUP BY month_of_crash_name ORDER BY CASE WHEN month_of_crash_name = 'January' THEN 1 WHEN month_of_crash_name = 'February' THEN 2 WHEN month_of_crash_name = 'March' THEN 3 WHEN month_of_crash_name = 'April' THEN 4 WHEN month_of_crash_name = 'May' THEN 5 WHEN month_of_crash_name = 'June' THEN 6 WHEN month_of_crash_name = 'July' THEN 7 WHEN month_of_crash_name = 'August' THEN 8 WHEN month_of_crash_name = 'September' THEN 9 WHEN month_of_crash_name = 'October' THEN 10 WHEN month_of_crash_name = 'November' THEN 11 WHEN month_of_crash_name = 'December' THEN 12 END"

months_avg_fatalities = pd.read_gbq(query_months_avg_fatalities, project_id=project_id, dialect='standard')

plt.bar(num_months, months_avg_fatalities["avg_fatalities"])

In [ ]:
%%bigquery --project $project_id
WITH state_months_accidents AS (
    SELECT COUNT(*) AS num_crashes, state_name, month_of_crash_name, RANK() OVER (PARTITION BY state_name ORDER BY COUNT(*) DESC) AS rank 
    FROM `traffic_fatalities.traffic_features` 
    
    GROUP BY state_name, state_number, month_of_crash_name ORDER BY rank
    ) 
SELECT w.num_crashes / t.total_accidents_per_state AS percent_accidents_per_year, w.month_of_crash_name, w.state_name, s.state 
FROM state_months_accidents AS w 
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT state_name, COUNT(*) AS total_accidents_per_state FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE rank = 1
ORDER BY state_name

In [ ]:
%%bigquery --project $project_id
WITH state_months_fatalities AS (
    SELECT SUM(number_of_fatalities) AS num_fatalities, state_name, month_of_crash_name, RANK() OVER (PARTITION BY state_name ORDER BY SUM(number_of_fatalities) DESC) AS rank 
    FROM `traffic_fatalities.traffic_features` 
    GROUP BY state_name, state_number, month_of_crash_name ORDER BY rank
    ) 
SELECT w.num_fatalities / t.total_fatalities_per_state AS percent_fatalities_per_year, w.month_of_crash_name, w.state_name, s.state 
FROM state_months_fatalities AS w 
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT state_name, SUM(number_of_fatalities) AS total_fatalities_per_state FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE rank = 1
ORDER BY state_name

In [ ]:

query_months_fatalities_state = "WITH state_months_accidents AS (SELECT SUM(number_of_fatalities) AS num_fatalities, state_name, month_of_crash_name, RANK() OVER (PARTITION BY state_name ORDER BY SUM(number_of_fatalities) DESC) AS rank FROM `traffic_fatalities.traffic_features` GROUP BY state_name, state_number, month_of_crash_name ORDER BY rank) SELECT w.num_fatalities AS num_fatalities_per_month, w.month_of_crash_name, w.state_name, s.state FROM state_months_accidents AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name WHERE rank = 1 ORDER BY CASE WHEN month_of_crash_name = 'January' THEN 1 WHEN month_of_crash_name = 'February' THEN 2 WHEN month_of_crash_name = 'March' THEN 3 WHEN month_of_crash_name = 'April' THEN 4 WHEN month_of_crash_name = 'May' THEN 5 WHEN month_of_crash_name = 'June' THEN 6 WHEN month_of_crash_name = 'July' THEN 7 WHEN month_of_crash_name = 'August' THEN 8 WHEN month_of_crash_name = 'September' THEN 9 WHEN month_of_crash_name = 'October' THEN 10 WHEN month_of_crash_name = 'November' THEN 11 WHEN month_of_crash_name = 'December' THEN 12 END"
months_fatalities_state = pd.read_gbq(query_months_fatalities_state, project_id=project_id, dialect='standard')

fig_months_fatalities_state = px.choropleth(
    months_fatalities_state,
    locations='state',
    locationmode='USA-states',
    color='month_of_crash_name',
    color_discrete_map = months_of_year_color_dict,
    title='Month with Most Fatalities',
)
fig_months_fatalities_state.update_layout(
    title_text = 'Month with Most Fatalities by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
WITH state_months_avg_fatalities AS (
    SELECT AVG(number_of_fatalities) AS avg_fatalities, state_name, month_of_crash_name, RANK() OVER (PARTITION BY state_name ORDER BY AVG(number_of_fatalities) DESC) AS rank 
    FROM `traffic_fatalities.traffic_features` 
    GROUP BY state_name, state_number, month_of_crash_name ORDER BY rank
    ) 
SELECT w.avg_fatalities, w.month_of_crash_name, w.state_name, s.state 
FROM state_months_avg_fatalities AS w 
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
WHERE rank = 1
ORDER BY state_name

In [ ]:
query_months_avg_fatalities_state = "WITH state_months_avg_fatalities AS (SELECT AVG(number_of_fatalities) AS avg_fatalities, state_name, month_of_crash_name, RANK() OVER (PARTITION BY state_name ORDER BY AVG(number_of_fatalities) DESC) AS rank FROM `traffic_fatalities.traffic_features` GROUP BY state_name, state_number, month_of_crash_name ORDER BY rank) SELECT w.avg_fatalities, w.month_of_crash_name, w.state_name, s.state FROM state_months_avg_fatalities AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name WHERE rank = 1 ORDER BY CASE WHEN month_of_crash_name = 'January' THEN 1 WHEN month_of_crash_name = 'February' THEN 2 WHEN month_of_crash_name = 'March' THEN 3 WHEN month_of_crash_name = 'April' THEN 4 WHEN month_of_crash_name = 'May' THEN 5 WHEN month_of_crash_name = 'June' THEN 6 WHEN month_of_crash_name = 'July' THEN 7 WHEN month_of_crash_name = 'August' THEN 8 WHEN month_of_crash_name = 'September' THEN 9 WHEN month_of_crash_name = 'October' THEN 10 WHEN month_of_crash_name = 'November' THEN 11 WHEN month_of_crash_name = 'December' THEN 12 END"
months_avg_fatalities_state = pd.read_gbq(query_months_avg_fatalities_state, project_id=project_id, dialect='standard')
fig_months_avg_fatalities_state = px.choropleth(
    months_avg_fatalities_state,
    locations='state',
    locationmode='USA-states',
    color='month_of_crash_name',
    color_discrete_map = months_of_year_color_dict,
    title='Month with Most Average per crash fatalities',
)

fig_months_avg_fatalities_state.update_layout(
    title_text = 'Month with Most Average fatalities per crash by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

## Drunk Drivers

In [ ]:
%%bigquery --project $project_id
SELECT SUM(number_of_drunk_drivers)/SUM(number_of_motor_vehicles_in_transport_mvit) * 100 as percent_drunk, COUNT(id) AS num_crashes, hour_of_crash_name 
FROM `traffic_fatalities.traffic_features` 
GROUP BY hour_of_crash_name
ORDER BY percent_drunk DESC

In [ ]:
# Percent of drunk drivers in fatal accidents by the hour with SEVERITY = 0
query_percent_drunk_hours  = "SELECT SUM(number_of_drunk_drivers)/SUM(number_of_motor_vehicles_in_transport_mvit) * 100 as percent_drunk, COUNT(id) AS num_crashes, hour_of_crash_name FROM `traffic_fatalities.traffic_features` GROUP BY hour_of_crash_name ORDER BY CASE WHEN hour_of_crash_name = '0:00am-0:59am' THEN 0 WHEN hour_of_crash_name = '1:00am-1:59am' THEN 1 WHEN hour_of_crash_name = '2:00am-2:59am' THEN 2 WHEN hour_of_crash_name = '3:00am-3:59am' THEN 3 WHEN hour_of_crash_name = '4:00am-4:59am' THEN 4 WHEN hour_of_crash_name = '5:00am-5:59am' THEN 5 WHEN hour_of_crash_name = '6:00am-6:59am' THEN 6 WHEN hour_of_crash_name = '7:00am-7:59am' THEN 7 WHEN hour_of_crash_name = '8:00am-8:59am' THEN 8 WHEN hour_of_crash_name = '9:00am-9:59am' THEN 9 WHEN hour_of_crash_name = '10:00am-10:59am' THEN 10 WHEN hour_of_crash_name = '11:00am-11:59am' THEN 11 WHEN hour_of_crash_name = '12:00pm-12:59pm' THEN 12 WHEN hour_of_crash_name = '1:00pm-1:59pm' THEN 13 WHEN hour_of_crash_name = '2:00pm-2:59pm' THEN 14 WHEN hour_of_crash_name = '3:00pm-3:59pm' THEN 15 WHEN hour_of_crash_name = '4:00pm-4:59pm' THEN 16 WHEN hour_of_crash_name = '5:00pm-5:59pm' THEN 17 WHEN hour_of_crash_name = '6:00pm-6:59pm' THEN 18 WHEN hour_of_crash_name = '7:00pm-7:59pm' THEN 19 WHEN hour_of_crash_name = '8:00pm-8:59pm' THEN 20 WHEN hour_of_crash_name = '9:00pm-9:59pm' THEN 21 WHEN hour_of_crash_name = '10:00pm-10:59pm' THEN 22 WHEN hour_of_crash_name = '11:00pm-11:59pm' THEN 23 END"

percent_drunk_hours = pd.read_gbq(query_percent_drunk_hours, project_id=project_id, dialect='standard')

plt.bar(num_hours, percent_drunk_hours["percent_drunk"])

In [ ]:
%%bigquery --project $project_id
SELECT SUM(number_of_drunk_drivers)/SUM(number_of_motor_vehicles_in_transport_mvit) * 100 as percent_drunk, COUNT(id) AS num_crashes, day_of_week_name 
FROM `traffic_fatalities.traffic_features` 
GROUP BY day_of_week_name
ORDER BY percent_drunk DESC

In [ ]:
# Percent of drunk drivers in fatal accidents by the hour with SEVERITY = 0
query_percent_drunk_days  = "SELECT SUM(number_of_drunk_drivers)/SUM(number_of_motor_vehicles_in_transport_mvit) * 100 as percent_drunk, COUNT(id) AS num_crashes, day_of_week_name FROM `traffic_fatalities.traffic_features` GROUP BY day_of_week_name ORDER BY CASE WHEN day_of_week_name = 'Moday' THEN 0 WHEN day_of_week_name = 'Tuesday' THEN 1 WHEN day_of_week_name = 'Wednesday' THEN 3 WHEN day_of_week_name = 'Thursday' THEN 4 WHEN day_of_week_name = 'Friday' THEN 5 WHEN day_of_week_name = 'Saturday' THEN 6 WHEN day_of_week_name = 'Sunday' THEN 7 END"

percent_drunk_days = pd.read_gbq(query_percent_drunk_days, project_id=project_id, dialect='standard')

plt.bar(percent_drunk_days["day_of_week_name"], percent_drunk_days["percent_drunk"])

In [ ]:
%%bigquery --project $project_id
SELECT SUM(number_of_drunk_drivers)/SUM(number_of_motor_vehicles_in_transport_mvit) * 100 as percent_drunk, COUNT(id) AS num_crashes, month_of_crash_name 
FROM `traffic_fatalities.traffic_features` 
GROUP BY month_of_crash_name
ORDER BY percent_drunk DESC

In [ ]:
# Percent of drunk drivers in fatal accidents by the hour with SEVERITY = 0
query_percent_drunk_months  = "SELECT SUM(number_of_drunk_drivers)/SUM(number_of_motor_vehicles_in_transport_mvit) * 100 as percent_drunk, COUNT(id) AS num_crashes, month_of_crash_name FROM `traffic_fatalities.traffic_features` GROUP BY month_of_crash_name ORDER BY CASE WHEN month_of_crash_name = 'January' THEN 1 WHEN month_of_crash_name = 'February' THEN 2 WHEN month_of_crash_name = 'March' THEN 3 WHEN month_of_crash_name = 'April' THEN 4 WHEN month_of_crash_name = 'May' THEN 5 WHEN month_of_crash_name = 'June' THEN 6 WHEN month_of_crash_name = 'July' THEN 7 WHEN month_of_crash_name = 'August' THEN 8 WHEN month_of_crash_name = 'September' THEN 9 WHEN month_of_crash_name = 'October' THEN 10 WHEN month_of_crash_name = 'November' THEN 11 WHEN month_of_crash_name = 'December' THEN 12 END"

percent_drunk_months = pd.read_gbq(query_percent_drunk_months, project_id=project_id, dialect='standard')

plt.bar(num_months, percent_drunk_months["percent_drunk"])

In [ ]:
%%bigquery --project $project_id
SELECT AVG(number_of_fatalities) AS avg_fatalities, number_of_drunk_drivers AS num_dd
FROM `traffic_fatalities.traffic_features` 
GROUP BY num_dd
ORDER BY num_dd

In [ ]:
# Percent of drunk drivers in fatal accidents by the hour with SEVERITY = 0
query_avg_dd  = "SELECT AVG(number_of_fatalities) AS avg_fatalities, number_of_drunk_drivers AS num_dd FROM `traffic_fatalities.traffic_features` GROUP BY num_dd ORDER BY num_dd"

avg_dd = pd.read_gbq(query_avg_dd, project_id=project_id, dialect='standard')

plt.bar(avg_dd["num_dd"], avg_dd["avg_fatalities"])

In [ ]:
%%bigquery --project $project_id

SELECT SUM(t.number_of_drunk_drivers)/SUM(t.number_of_motor_vehicles_in_transport_mvit) * 100 as percent_drunk, COUNT(*) AS num_crashes, t.state_name, s.state
FROM `traffic_fatalities.traffic_features` AS t
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON t.state_name = s.state_name 
GROUP BY state_name, state
ORDER BY percent_drunk DESC

In [ ]:
query_percent_drunk_drivers_per_state = "SELECT SUM(t.number_of_drunk_drivers)/SUM(t.number_of_motor_vehicles_in_transport_mvit) * 100 as percent_drunk, COUNT(*) AS num_crashes, t.state_name, s.state FROM `traffic_fatalities.traffic_features` AS t JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON t.state_name = s.state_name GROUP BY state_name, state"
percent_drunk_drivers_per_state = pd.read_gbq(query_percent_drunk_drivers_per_state, project_id=project_id, dialect='standard')
fig_percent_drunk_drivers_per_state = px.choropleth(
    percent_drunk_drivers_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_drunk',
    color_continuous_scale = 'Reds',
    title='Percent Drunk Drivers by State',
)

fig_percent_drunk_drivers_per_state.update_layout(
    title_text = 'Percent Drunk Drivers by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

### Atmospheric Conditions

In [ ]:
%%bigquery --project $project_id
SELECT AVG(number_of_fatalities) AS avg_fatalities, COUNT(*) AS num_accidents, atmospheric_conditions_name
FROM `traffic_fatalities.traffic_features` 
GROUP BY atmospheric_conditions_name
ORDER BY avg_fatalities

In [ ]:
# Percent of drunk drivers in fatal accidents by the hour with SEVERITY = 0
query_avg_condtions  = "SELECT AVG(number_of_fatalities) AS avg_fatalities, COUNT(*) AS num_accidents, atmospheric_conditions_name FROM `traffic_fatalities.traffic_features` GROUP BY atmospheric_conditions_name"

avg_condtions = pd.read_gbq(query_avg_condtions, project_id=project_id, dialect='standard')

plt.barh(avg_condtions["atmospheric_conditions_name"], avg_condtions["avg_fatalities"])
# Set labels and title
plt.xlabel('Average Fatalities')
plt.ylabel('Atmospheric Conditions')
plt.title('Average Fatalities by Atmospheric Conditions')

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_sand_soil_dirt, COUNT(*) AS num_sand_soil_dirt, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Blowing Sand, Soil, Dirt'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_dirt_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_sand_soil_dirt, COUNT(*) AS num_sand_soil_dirt, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Blowing Sand, Soil, Dirt' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
dirt_per_state = pd.read_gbq(query_dirt_per_state, project_id=project_id, dialect='standard')
fig_dirt_per_state = px.choropleth(
    dirt_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_sand_soil_dirt',
    color_continuous_scale = 'Reds',
)

fig_dirt_per_state.update_layout(
    title_text = 'Percent of Accidents that have Blowing Sand, Soil, Dirt by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_dirt_avg_fatalities = px.choropleth(
    dirt_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_dirt_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that have Blowing Sand, Soil, Dirt by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_sleet_hail, COUNT(*) AS num_sleet_hail, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Sleet or Hail'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_sleet_or_hail_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_sleet_hail, COUNT(*) AS num_sleet_hail, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Sleet or Hail' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
sleet_or_hail_per_state = pd.read_gbq(query_sleet_or_hail_per_state, project_id=project_id, dialect='standard')
fig_sleet_or_hail_per_state = px.choropleth(
    sleet_or_hail_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_sleet_hail',
    color_continuous_scale = 'Reds',
)

fig_sleet_or_hail_per_state.update_layout(
    title_text = 'Percent of Accidents that have Sleet Or Hail by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_sleet_or_hail_avg_fatalities = px.choropleth(
    sleet_or_hail_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_sleet_or_hail_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that have Sleet or Hail by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_fog_smog_smoke, COUNT(*) AS num_fog_smog_smoke, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Fog, Smog, Smoke'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_fog_smog_smoke_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_fog_smog_smoke, COUNT(*) AS num_fog_smog_smoke, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Fog, Smog, Smoke' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
fog_smog_smoke_per_state = pd.read_gbq(query_fog_smog_smoke_per_state, project_id=project_id, dialect='standard')
fig_fog_smog_smoke_per_state = px.choropleth(
    fog_smog_smoke_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_fog_smog_smoke',
    color_continuous_scale = 'Reds',
)

fig_fog_smog_smoke_per_state.update_layout(
    title_text = 'Percent of Accidents that have Fog, Smog, Or Smoke by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_fog_smog_smoke_avg_fatalities = px.choropleth(
    fog_smog_smoke_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_fog_smog_smoke_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that have Fog, Smog, or Smoke by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_rain, COUNT(*) AS num_rain, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Rain'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_rain_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_rain, COUNT(*) AS num_rain, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Rain' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
rain_per_state = pd.read_gbq(query_rain_per_state, project_id=project_id, dialect='standard')
fig_rain_per_state = px.choropleth(
    rain_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_rain',
    color_continuous_scale = 'Reds',
)

fig_rain_per_state.update_layout(
    title_text = 'Percent of Accidents that have Rain by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_rain_avg_fatalities = px.choropleth(
    rain_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_rain_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that have Rain by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_crosswind, COUNT(*) AS num_crosswind, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Severe Crosswinds'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_crosswind_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_crosswind, COUNT(*) AS num_crosswind, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Severe Crosswinds' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
crosswind_per_state = pd.read_gbq(query_crosswind_per_state, project_id=project_id, dialect='standard')
fig_crosswind_per_state = px.choropleth(
    crosswind_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_crosswind',
    color_continuous_scale = 'Reds',
)

fig_crosswind_per_state.update_layout(
    title_text = 'Percent of Accidents that have Severe Crosswinds by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_crosswind_avg_fatalities = px.choropleth(
    crosswind_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_crosswind_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that have Severe Crosswinds by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_freezing_rain_drizzle, COUNT(*) AS num_freezing_rain_drizzle, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Freezing Rain or Drizzle'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_freezing_rain_drizzle_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_freezing_rain_drizzle, COUNT(*) AS num_freezing_rain_drizzle, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Freezing Rain or Drizzle' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
freezing_rain_drizzle_per_state = pd.read_gbq(query_freezing_rain_drizzle_per_state, project_id=project_id, dialect='standard')
fig_freezing_rain_drizzle_per_state = px.choropleth(
    freezing_rain_drizzle_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_freezing_rain_drizzle',
    color_continuous_scale = 'Reds',
)

fig_freezing_rain_drizzle_per_state.update_layout(
    title_text = 'Percent of Accidents that have Freezing Rain or Drizzle by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_freezing_rain_drizzle_avg_fatalities = px.choropleth(
    freezing_rain_drizzle_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_freezing_rain_drizzle_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that have Freezing Rain or Drizzle by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_snow, COUNT(*) AS num_snow, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Snow'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_snow_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_snow, COUNT(*) AS num_snow, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Snow' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
snow_per_state = pd.read_gbq(query_snow_per_state, project_id=project_id, dialect='standard')
fig_snow_per_state = px.choropleth(
    snow_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_snow',
    color_continuous_scale = 'Reds',
)

fig_snow_per_state.update_layout(
    title_text = 'Percent of Accidents that have Snow by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_snow_avg_fatalities = px.choropleth(
    snow_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_snow_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that have Snow by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_cloudy, COUNT(*) AS num_cloudy, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Cloudy'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_cloudy_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_cloudy, COUNT(*) AS num_cloudy, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Cloudy' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
cloudy_per_state = pd.read_gbq(query_cloudy_per_state, project_id=project_id, dialect='standard')
fig_cloudy_per_state = px.choropleth(
    cloudy_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_cloudy',
    color_continuous_scale = 'Reds',
)

fig_cloudy_per_state.update_layout(
    title_text = 'Percent of Accidents that occur while Cloudy by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_cloudy_avg_fatalities = px.choropleth(
    cloudy_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_cloudy_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that occur while Cloudy by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_clear, COUNT(*) AS num_clear, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Clear'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_clear_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_clear, COUNT(*) AS num_clear, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Clear' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
clear_per_state = pd.read_gbq(query_clear_per_state, project_id=project_id, dialect='standard')
fig_clear_per_state = px.choropleth(
    clear_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_clear',
    color_continuous_scale = 'Reds',
)

fig_clear_per_state.update_layout(
    title_text = 'Percent of Accidents that occur while Clear by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_clear_avg_fatalities = px.choropleth(
    clear_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_clear_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that occur while Clear by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_blowing_snow, COUNT(*) AS num_blowing_snow, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state
FROM `traffic_fatalities.traffic_features` AS w
JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name 
JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name
WHERE atmospheric_conditions_name = 'Blowing Snow'
GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state

In [ ]:
query_blowing_snow_per_state = "SELECT COUNT(*)/t.total_accidents_per_state * 100 AS percent_blowing_snow, COUNT(*) AS num_clear, AVG(w.number_of_fatalities) AS avg_fatalities, w.state_name, w.atmospheric_conditions_name, s.state FROM `traffic_fatalities.traffic_features` AS w JOIN `bigquery-public-data.geo_us_boundaries.states` AS s ON w.state_name = s.state_name JOIN (SELECT COUNT(*) AS total_accidents_per_state, state_name FROM `traffic_fatalities.traffic_features` GROUP BY state_name) AS t ON w.state_name = t.state_name WHERE atmospheric_conditions_name = 'Blowing Snow' GROUP BY state_name, atmospheric_conditions_name, state, total_accidents_per_state"
blowing_snow_per_state = pd.read_gbq(query_blowing_snow_per_state, project_id=project_id, dialect='standard')
fig_blowing_snow_per_state = px.choropleth(
    blowing_snow_per_state,
    locations='state',
    locationmode='USA-states',
    color='percent_blowing_snow',
    color_continuous_scale = 'Reds',
)

fig_blowing_snow_per_state.update_layout(
    title_text = 'Percent of Accidents that have Blowing Snow by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

In [ ]:
fig_blowing_snow_avg_fatalities = px.choropleth(
    blowing_snow_per_state,
    locations='state',
    locationmode='USA-states',
    color='avg_fatalities',
    color_continuous_scale = 'Blues',
)
fig_blowing_snow_avg_fatalities.update_layout(
    title_text = 'Average Fatalities in Accidents that have Blowing Snow by State 2015-2020',
    geo_scope='usa', # limite map scope to USA
)

## Number of Fatalities Ranked by Average time to Scene

In [ ]:
%%bigquery --project $project_id

SELECT number_of_fatalities, COUNT(number_of_fatalities) AS count, AVG(time_to_scene) AS avg_time_to_scene
FROM `traffic_fatalities.traffic_features` 
WHERE time_to_scene != 9999
GROUP BY number_of_fatalities
ORDER BY number_of_fatalities

## Number of Fatalities per state Ranked by Average Time to Scene

In [ ]:
%%bigquery --project $project_id

SELECT state_name, SUM(number_of_fatalities) AS total_fatalities, COUNT(state_name) AS num_crashes, AVG(time_to_scene) AS avg_time_to_scene
FROM (SELECT DISTINCT id, state_name, number_of_fatalities, time_to_scene FROM `traffic_fatalities.traffic_features` WHERE time_to_scene != 9999)
GROUP BY state_name
ORDER BY avg_time_to_scene

In [ ]:
%%bigquery --project $project_id

SELECT state_name, SUM(number_of_fatalities)/(SUM(number_of_persons_in_motor_vehicles_in_transport_mvit) + SUM(number_of_persons_not_in_motor_vehicles_in_transport_mvit)) AS percent_fatalities, COUNT(*) AS num_crashes, AVG(time_to_scene) AS avg_time_to_scene
FROM `traffic_fatalities.traffic_features` 
WHERE time_to_scene != 9999
GROUP BY state_name
ORDER BY avg_time_to_scene

## Number of Fatalities per State Ranked by Average Time to Hospital

In [ ]:
%%bigquery --project $project_id

SELECT state_name, SUM(number_of_fatalities) AS total_fatalities, COUNT(state_name) AS num_crashes, AVG(time_to_hospital) AS avg_time_to_hospital
FROM (SELECT DISTINCT id, state_name, number_of_fatalities, time_to_hospital FROM `traffic_fatalities.traffic_features` WHERE time_to_hospital != 9999)
GROUP BY state_name
ORDER BY avg_time_to_hospital

## Geographic Analysis

In [ ]:
# This shows differences in frequency of high severity crashes across states
%%bigquery --project $project_id
SELECT state_name, count(*)
FROM (SELECT DISTINCT id, state_name, number_of_fatalities, time_to_hospital FROM `traffic_fatalities.traffic_features` WHERE time_to_hospital != 9999)
GROUP BY state_name
ORDER BY avg_time_to_hospital

In [ ]:
query  = "SELECT COUNT(DISTINCT(id)) as num_crashes, city_name FROM `traffic_fatalities.traffic_features` WHERE city_name != 'NOT APPLICABLE' AND city_name != 'Not Reported' GROUP BY city_name ORDER BY num_crashes DESC LIMIT 10"
city = pd.read_gbq(query, project_id=project_id, dialect='standard')
plt.bar(city["city_name"], city["num_crashes"])
city

## Baseline and Data Prediction

### Training Time :D

In [ ]:
%%bigquery --project $project_id
SELECT COUNT(*)
FROM `traffic_fatalities.traffic_features`

In [ ]:
%%bigquery --project $project_id

CREATE OR REPLACE TABLE `traffic_fatalities.traffic_rand_data` AS
SELECT
    *,
    ROW_NUMBER() OVER(ORDER BY RAND()) AS rand_id
FROM `traffic_fatalities.traffic_features`

In [ ]:
%%bigquery --project $project_id

# YOUR QUERY HERE
CREATE OR REPLACE MODEL `traffic_fatalities.traffic_model`
OPTIONS(model_type='logistic_reg') AS
SELECT 
    state_name,
    number_of_motor_vehicles_in_transport_mvit,
    number_of_parked_working_vehicles,
    number_of_persons_in_motor_vehicles_in_transport_mvit,
    number_of_persons_not_in_motor_vehicles_in_transport_mvit,
    day_name,
    month_of_crash_name,
    day_of_week_name,
    hour_of_crash_name,
    route_signing_name,
    land_use_name,
    first_harmful_event_name,
    relation_to_trafficway_name,
    light_condition_name,
    atmospheric_conditions_name,
    time_to_scene,
    time_to_hospital,
    number_of_drunk_drivers,
    label,
FROM `traffic_fatalities.traffic_rand_data`
WHERE rand_id <= 136942

In [ ]:
%%bigquery --project $project_id

# Run cell to view training stats

SELECT
  *
FROM
  ML.TRAINING_INFO(MODEL `traffic_fatalities.traffic_model`)

In [ ]:
%%bigquery --project $project_id
# YOUR QUERY HERE


SELECT
  *
FROM
  ML.EVALUATE(MODEL `traffic_fatalities.traffic_model`, (
      SELECT
        state_name,
        number_of_motor_vehicles_in_transport_mvit,
        number_of_parked_working_vehicles,
        number_of_persons_in_motor_vehicles_in_transport_mvit,
        number_of_persons_not_in_motor_vehicles_in_transport_mvit,
        day_name,
        month_of_crash_name,
        day_of_week_name,
        hour_of_crash_name,
        route_signing_name,
        land_use_name,
        first_harmful_event_name,
        relation_to_trafficway_name,
        light_condition_name,
        atmospheric_conditions_name,
        time_to_scene,
        time_to_hospital,
        number_of_drunk_drivers,
        label,
      FROM `traffic_fatalities.traffic_rand_data`
      WHERE rand_id > 136942 AND rand_id <= 154060))

In [ ]:
%%bigquery --project $project_id

# YOUR QUERY HERE
SELECT
  *
FROM
  ML.EVALUATE(MODEL `traffic_fatalities.traffic_model`, (
      SELECT
        state_name,
        number_of_motor_vehicles_in_transport_mvit,
        number_of_parked_working_vehicles,
        number_of_persons_in_motor_vehicles_in_transport_mvit,
        number_of_persons_not_in_motor_vehicles_in_transport_mvit,
        day_name,
        month_of_crash_name,
        day_of_week_name,
        hour_of_crash_name,
        route_signing_name,
        land_use_name,
        first_harmful_event_name,
        relation_to_trafficway_name,
        light_condition_name,
        atmospheric_conditions_name,
        time_to_scene,
        time_to_hospital,
        number_of_drunk_drivers,
        label,
      FROM `traffic_fatalities.traffic_rand_data`
      WHERE rand_id > 154060
  ))

In [ ]:
%%bigquery --project $project_id

# YOUR QUERY HERE
SELECT
  state_name,
  number_of_motor_vehicles_in_transport_mvit,
  number_of_parked_working_vehicles,
  number_of_persons_in_motor_vehicles_in_transport_mvit,
  number_of_persons_not_in_motor_vehicles_in_transport_mvit,
  day_name,
  month_of_crash_name,
  day_of_week_name,
  hour_of_crash_name,
  route_signing_name,
  land_use_name,
  first_harmful_event_name,
  relation_to_trafficway_name,
  light_condition_name,
  atmospheric_conditions_name,
  time_to_scene,
  time_to_hospital,
  number_of_drunk_drivers,
  predicted_label
FROM
  ML.PREDICT(MODEL `traffic_fatalities.traffic_model`, (
  SELECT
    state_name,
    number_of_motor_vehicles_in_transport_mvit,
    number_of_parked_working_vehicles,
    number_of_persons_in_motor_vehicles_in_transport_mvit,
    number_of_persons_not_in_motor_vehicles_in_transport_mvit,
    day_name,
    month_of_crash_name,
    day_of_week_name,
    hour_of_crash_name,
    route_signing_name,
    land_use_name,
    first_harmful_event_name,
    relation_to_trafficway_name,
    light_condition_name,
    atmospheric_conditions_name,
    time_to_scene,
    time_to_hospital,
    number_of_drunk_drivers,
  FROM `traffic_fatalities.traffic_features`
  WHERE time_to_scene <= 5 AND number_of_motor_vehicles_in_transport_mvit = 2 AND number_of_persons_in_motor_vehicles_in_transport_mvit > 3))
LIMIT 10

## Conclusion

---

*TODO: Final conclusions based on the rest of your project*

---